In [ ]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [23]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model2 = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=GOOGLE_API_KEY,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [4]:
function_list =[
    {
        "type": "function",
        "function": {
            "name": "gastos_consulta",
            "description": "Utiliza esta función para obtener un resumen de los gastos registrados en un mes y año específicos, y opcionalmente por categoría.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mes": {
                        "type": "string",
                        "description": "El mes para el cual se consultan los gastos. Por ejemplo: 'enero', 'febrero'."
                    },
                    "año": {
                        "type": "string",
                        "description": "El año para el cual se consultan los gastos. Por ejemplo: '2024'."
                    },
                    "categoria":{
                        "type": "string",
                        "description": "Es la categoría de clasificación del casto. Por ejemplo: 'abonos'"
                    },
                },
                "required": ["mes", "año"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "gastos_agregar",
            "description": "Registra un nuevo gasto. Es necesario especificar el valor, la fecha (mes y año) y opcionalmente una categoría.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mes": {
                        "type": "string",
                        "description": "El mes en que se realizó el gasto. Por ejemplo: 'enero'."
                    },
                    "año": {
                        "type": "string",
                        "description": "El año en que se realizó el gasto. Por ejemplo: '2024'."
                    },
                    "valor": {
                        "type": "number",
                        "description": "El monto o valor monetario del gasto."
                    },
                    "categoria": {
                        "type": "string",
                        "description": "Una categoría para clasificar el gasto. Por ejemplo: 'herramientas', 'transporte', 'insumos'. Este campo es opcional."
                    }
                },
                "required": ["mes", "año", "valor"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "ingresos_consulta",
            "description": "Utiliza esta función para obtener un resumen de los ingresos registrados en un mes y año específicos.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mes": {
                        "type": "string",
                        "description": "El mes para el cual se consultan los ingresos. Por ejemplo: 'marzo'."
                    },
                    "año": {
                        "type": "string",
                        "description": "El año para el cual se consultan los ingresos. Por ejemplo: '2025'."
                    }
                },
                "required": ["mes", "año"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "ingresos_agregar",
            "description": "Registra un nuevo ingreso. Es necesario especificar el valor, la fecha (mes y año)",
            "parameters": {
                "type": "object",
                "properties": {
                    "mes": {
                        "type": "string",
                        "description": "El mes en que se recibió el ingreso. Por ejemplo: 'marzo'."
                    },
                    "año": {
                        "type": "string",
                        "description": "El año en que se recibió el ingreso. Por ejemplo: '2025'."
                    },
                    "valor": {
                        "type": "number",
                        "description": "El monto o valor monetario del ingreso."
                    }
                },
                "required": ["mes", "año", "valor"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "produccion_consulta",
            "description": "Consulta y retorna la cantidad de producción esperada para un lote agrícola específico.",
            "parameters": {
                "type": "object",
                "properties": {
                    "lote": {
                        "type": "string",
                        "description": "El identificador o nombre del lote a consultar. Por ejemplo: 'LoteA', 'LoteB'."
                    }
                },
                "required": ["lote"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "inventario_consulta",
            "description": "Retorna la cantidad de unidades disponibles en inventario para un producto específico.",
            "parameters": {
                "type": "object",
                "properties": {
                    "producto": {
                        "type": "string",
                        "description": "El nombre del producto o insumo a consultar en el inventario. Por ejemplo: 'fertilizante', 'semillas'."
                    }
                },
                "required": ["producto"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "inventario_actualizar",
            "description": "Actualiza el inventario de un producto. Se debe especificar la cantidad y si es una entrada o una salida.",
            "parameters": {
                "type": "object",
                "properties": {
                    "producto": {
                        "type": "string",
                        "description": "El nombre del producto cuyo inventario se va a actualizar."
                    },
                    "cantidad": {
                        "type": "number",
                        "description": "La cantidad a modificar en el inventario. Usa un número positivo para agregar (entrada) y un número negativo para quitar (salida)."
                    }
                },
                "required": ["producto", "cantidad"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "cosecha_actualizar",
            "description": "Registra una nueva cosecha, especificando la cantidad en kilogramos (kg) para un mes y año determinados.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mes": {
                        "type": "string",
                        "description": "El mes en que se realizó la cosecha."
                    },
                    "año": {
                        "type": "string",
                        "description": "El año en que se realizó la cosecha."
                    },
                    "cantidad": {
                        "type": "number",
                        "description": "La cantidad total cosechada, expresada en kilogramos (kg)."
                    }
                },
                "required": ["año", "mes", "cantidad"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "cosecha_consultar",
            "description": "Consulta la cantidad total de cosecha (en kg) registrada para un mes y año específicos.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mes": {
                        "type": "string",
                        "description": "El mes de la cosecha a consultar."
                    },
                    "año": {
                        "type": "string",
                        "description": "El año de la cosecha a consultar."
                    }
                },
                "required": ["año", "mes"]
            }
        }
    }
]

In [7]:
function_descriptions = {
    "gastos_consulta": {
        "description": "Consultar gastos por mes/año y opcionalmente categoría",
        "params": ["mes", "año", "categoria"],
        "required": ["mes", "año"]
    },
    "gastos_agregar": {
        "description": "Registrar un nuevo gasto con valor, mes/año y opcionalmente categoría",
        "params": ["mes", "año", "valor", "categoria"],
        "required": ["mes", "año", "valor"]
    },
    "ingresos_consulta": {
        "description": "Consultar ingresos por mes/año",
        "params": ["mes", "año"],
        "required": ["mes", "año"]
    },
    "ingresos_agregar": {
        "description": "Registrar un nuevo ingreso con valor y mes/año",
        "params": ["mes", "año", "valor"],
        "required": ["mes", "año", "valor"]
    },
    "produccion_consulta": {
        "description": "Consultar producción esperada de un lote específico",
        "params": ["lote"],
        "required": ["lote"]
    },
    "inventario_consulta": {
        "description": "Consultar cantidad en inventario de un producto",
        "params": ["producto"],
        "required": ["producto"]
    },
    "inventario_actualizar": {
        "description": "Actualizar inventario (positivo=entrada, negativo=salida)",
        "params": ["producto", "cantidad"],
        "required": ["producto", "cantidad"]
    },
    "cosecha_actualizar": {
        "description": "Registrar nueva cosecha en kg para mes/año",
        "params": ["mes", "año", "cantidad"],
        "required": ["mes", "año", "cantidad"]
    },
    "cosecha_consultar": {
        "description": "Consultar cosecha en kg por mes/año",
        "params": ["mes", "año"],
        "required": ["mes", "año"]
    }
}

In [8]:
import pandas as pd
import json
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Dict
import time
import time
from datetime import datetime

In [15]:
prompt_template =  """
Tu tarea es generar un par de datos para entrenar un modelo de 'function calling'.
Debes crear una pregunta de usuario en español y la correspondiente llamada a función en formato JSON.

**Función a utilizar:**
```json
{function_json}
Instrucciones:

Inventa una pregunta de usuario que sea realista, natural y que claramente intente ejecutar la acción descrita en la función.

La pregunta debe contener toda la información necesaria para llenar los parámetros de la función.

Crea la llamada a la función como un string que contenga un diccionario con 'name' y 'arguments'.

Genera valores creativos y diversos para los parámetros (meses, años, montos, nombres, etc.).

El formato de tu respuesta DEBE ser el siguiente, sin explicaciones adicionales:

Query: [Aquí la pregunta que generaste]
Function: [Aquí el diccionario de la función como string]

Ejemplo de formato de salida:
Query: podrías registrar un gasto de 120.500 en transporte para julio del 2024
Function: {{'name': 'gastos_agregar', 'arguments': '{{"mes": "julio", "año": "2024", "valor": 120500.50, "categoria": "transporte"}}'}}

Ahora, genera un nuevo ejemplo para la función proporcionada.
"""

The next script generates the questions based on the functions above

In [ ]:
import pandas as pd
import json
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import time
import random
from datetime import datetime


system_template = """Eres un asistente que genera preguntas naturales en español para un sistema de gestión agrícola.
Genera preguntas MUY variadas y realistas que un agricultor haría.

IMPORTANTE: Cada pregunta debe ser ÚNICA. Varía:
- La forma de formular (formal, informal, con errores de tipeo ocasionales)
- Los valores específicos
- El orden de las palabras
- Usa sinónimos y diferentes expresiones
- Algunas con signos de interrogación, otras sin ellos

Valores numéricos:
- Gastos/Ingresos: 1000 a 10000000
- Inventario: 1 a 5000 unidades
- Cosecha: 50 a 50000 kg

Categorías de gastos: herramientas, transporte, insumos, fertilizantes, semillas, mano de obra, 
maquinaria, servicios, combustible, mantenimiento, agua, electricidad, impuestos, seguros, alquiler

Productos: fertilizante, semillas, pesticidas, abono, combustible, sacos, cajas, herbicidas, 
fungicidas, cal, azufre, alambre, postes, mangueras, aspersores, guantes, botas

Lotes: LoteA, LoteB, LoteC, Lote1, Lote2, Lote3, LoteNorte, LoteSur, Parcela1, Campo1, Invernadero1

Meses: todos los meses del año
Años: 2020-2026"""

def procesar_batch(args):
    function_name, batch_num, n_questions, total_batches = args
    
    print(f"  Procesando {function_name} batch {batch_num}/{total_batches} ({n_questions} preguntas)...")
    
    func_info = function_descriptions[function_name]
    
    user_prompt = f"""Genera EXACTAMENTE {n_questions} preguntas ÚNICAS para la función '{function_name}'.
    
    Descripción: {func_info["description"]}
    Parámetros: {", ".join(func_info["params"])}
    Requeridos: {", ".join(func_info["required"])}
    
    Este es el batch #{batch_num}, genera preguntas diferentes a los anteriores.
    
    Formato JSON:
    [
        {{
            "query": "pregunta en español",
            "arguments": {{"param1": "valor1", "param2": "valor2"}}
        }}
    ]"""
    
    messages = [
        {"role": "system", "content": system_template},
        {"role": "user", "content": user_prompt}
    ]
    
    for intento in range(3):
        try:
            if batch_num % 20 == 0 and batch_num > 0:
                print(f"    Pausa de 30s después de {batch_num} batches...")
                time.sleep(30)
            
            response = model.invoke(messages)
            content = response.content
            
            questions = json.loads(content)
            formatted_questions = []
            
            for q in questions:
                formatted_q = {
                    "query": q["query"],
                    "function": json.dumps({
                        "name": function_name,
                        "arguments": json.dumps(q["arguments"], ensure_ascii=False)
                    }, ensure_ascii=False)
                }
                formatted_questions.append(formatted_q)
            
            print(f"    ✓ {function_name} batch {batch_num} OK con OpenAI")
            return function_name, batch_num, formatted_questions
            
        except Exception as e:
            if "rate_limit" in str(e).lower() or "429" in str(e):
                wait_time = (2 ** intento) * 5 + random.uniform(1, 5)
                print(f"    Rate limit OpenAI, esperando {wait_time:.1f}s...")
                time.sleep(wait_time)
            else:
                print(f"    Error OpenAI: {str(e)[:100]}")
                break
    
    print(f"    Usando Gemini para {function_name} batch {batch_num}...")
    for intento in range(3):
        try:
            response = model2.invoke(messages)
            content = response.content
            
            questions = json.loads(content)
            formatted_questions = []
            
            for q in questions:
                formatted_q = {
                    "query": q["query"],
                    "function": json.dumps({
                        "name": function_name,
                        "arguments": json.dumps(q["arguments"], ensure_ascii=False)
                    }, ensure_ascii=False)
                }
                formatted_questions.append(formatted_q)
            
            print(f"    ✓ {function_name} batch {batch_num} OK con Gemini")
            return function_name, batch_num, formatted_questions
            
        except Exception as e:
            if "rate_limit" in str(e).lower() or "429" in str(e):
                wait_time = (2 ** intento) * 3 + random.uniform(1, 3)
                print(f"    Rate limit Gemini, esperando {wait_time:.1f}s...")
                time.sleep(wait_time)
            else:
                print(f"    Error Gemini: {str(e)[:100]}")
    
    print(f"    ❌ {function_name} batch {batch_num} falló")
    return function_name, batch_num, []

pairs_per_function = 300
batch_size = 25

os.makedirs("qa_pairs_output", exist_ok=True)

print(f"\n🚀 Generando {pairs_per_function * len(function_descriptions)} preguntas totales")
print(f"📊 {pairs_per_function} preguntas por cada una de las {len(function_descriptions)} funciones")
print(f"🔧 Batches de {batch_size} preguntas\n")

start_time = datetime.now()

all_tasks = []
for func_name in function_descriptions.keys():
    total_batches = (pairs_per_function + batch_size - 1) // batch_size
    
    for batch_start in range(0, pairs_per_function, batch_size):
        batch_end = min(batch_start + batch_size, pairs_per_function)
        batch_n = batch_end - batch_start
        batch_num = (batch_start // batch_size) + 1
        
        task = (func_name, batch_num, batch_n, total_batches)
        all_tasks.append(task)

print(f"📋 Total de batches: {len(all_tasks)}")

all_results = {func: [] for func in function_descriptions.keys()}
completed = 0

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = {executor.submit(procesar_batch, task): task for task in all_tasks}
    
    for future in as_completed(futures):
        function_name, batch_num, results = future.result()
        completed += 1
        
        if results:
            all_results[function_name].extend(results)
            print(f"[{completed}/{len(all_tasks)}] ✅ {function_name} batch {batch_num}: {len(results)} preguntas")
        else:
            print(f"[{completed}/{len(all_tasks)}] ❌ {function_name} batch {batch_num}: FALLÓ")
        
        if completed % 20 == 0:
            df_temp = pd.DataFrame([item for sublist in all_results.values() for item in sublist])
            df_temp.to_csv(f"qa_pairs_output/progreso_{completed}.csv", index=False)
            print(f"💾 Guardado progreso: {len(df_temp)} preguntas")

all_pairs = []
print("\n📊 Resumen por función:")
for func_name, pairs in all_results.items():
    all_pairs.extend(pairs)
    print(f"{func_name}: {len(pairs)} preguntas")

df = pd.DataFrame(all_pairs)
df = df.sample(frac=1).reset_index(drop=True)


🚀 Generando 2700 preguntas totales
📊 300 preguntas por cada una de las 9 funciones
🔧 Batches de 25 preguntas

📋 Total de batches: 108
  Procesando gastos_consulta batch 1/12 (25 preguntas)...
  Procesando gastos_consulta batch 2/12 (25 preguntas)...
  Procesando gastos_consulta batch 3/12 (25 preguntas)...
    ✓ gastos_consulta batch 1 OK con OpenAI
  Procesando gastos_consulta batch 4/12 (25 preguntas)...
[1/108] ✅ gastos_consulta batch 1: 25 preguntas
    ✓ gastos_consulta batch 3 OK con OpenAI
  Procesando gastos_consulta batch 5/12 (25 preguntas)...
[2/108] ✅ gastos_consulta batch 3: 25 preguntas
    ✓ gastos_consulta batch 4 OK con OpenAI
  Procesando gastos_consulta batch 6/12 (25 preguntas)...
[3/108] ✅ gastos_consulta batch 4: 25 preguntas
    ✓ gastos_consulta batch 2 OK con OpenAI
  Procesando gastos_consulta batch 7/12 (25 preguntas)...
[4/108] ✅ gastos_consulta batch 2: 25 preguntas
    ✓ gastos_consulta batch 5 OK con OpenAI
  Procesando gastos_consulta batch 8/12 (25 pre

In [31]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"qa_pairs_agricultura_{len(df)}_{timestamp}.csv"
df.to_csv(filename, index=False, encoding='utf-8')

In [32]:
df.head()

,query,function
0,Consulta de ingresos: septiembre 2025,"{""name"": ""ingresos_consulta"", ""arguments"": ""{\..."
1,quiero saber kg de cosecha del mes diciembre d...,"{""name"": ""cosecha_consultar"", ""arguments"": ""{\..."
2,Anota gasto de 800000 correspondiente a diciem...,"{""name"": ""gastos_agregar"", ""arguments"": ""{\""me..."
3,"LoteSur, cuál es la produccion calculada?","{""name"": ""produccion_consulta"", ""arguments"": ""..."
4,aumentar cosecha a 2396 kg en marzo 2023,"{""name"": ""cosecha_actualizar"", ""arguments"": ""{..."


In [33]:
df.shape

(2702, 2)

In [28]:
df.head(1)

,query,function
0,¿Tienes registrado cuántas botas quedan?,"{""name"": ""inventario_consulta"", ""arguments"": ""..."


In [29]:
df['function'][0]

'{"name": "inventario_consulta", "arguments": "{\\"producto\\": \\"botas\\"}"}'